In [89]:

import pymongo
from pymongo import MongoClient
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
from bson.son import SON
from bson.code import Code
import json
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3


In [3]:
client = MongoClient()
#creating mbf database (mbf = "meta beauty file")
mbf = client.dsbc.mbf
# mbf.remove()


In [ ]:
with open('meta_Beauty.json') as meta_beauty_file:
    for line in meta_beauty_file:
        meta_f = json.loads(line)  
        #only work when using json.loads() not json.load() 
        mbf.insert(meta_f)
        
#can't do json.load(meta_beautfy_file) since meta_beauty_file has the form "{...}{...}{...}" --> not json
#json.load(line) does not work since line is  string

In [4]:
# "categories": [["Beauty", "Skin Care",...]] --> need to $unwind
# use "categories.0" get the innner list.

#Skin_care = mbf.find({"categories.0": ["Beauty"]})

pipeline  = [{"$unwind":"$categories"},
             {"$match":{"categories":{"$all":["Face","Creams & Moisturizers"]}}},
             {"$project":{"_id":0,"asin":1,"title":1,"price":1}},
             {"$out": "product_info"}]
# "$out" write the result into new collection
# "$all" find all list that contains everything in queries
mbf.aggregate(pipeline)

In [6]:
product_info = client.dsbc.product_info

In [7]:
# name = product_info.find({}, {"_id":0,"asin":1})
# id_list  = {}
# for id_num in name:
#     id_list.append(id_num["asin"])

In [8]:
name = product_info.find({}, {"_id":0,"asin":1,"title":1,"price":1})
id_list  = {}
for id_num in name:
    try:
        id_list[id_num["asin"]]=[id_num["title"],id_num["price"]]
    except:
        pass


In [9]:
len(id_list.keys())

5570

In [10]:
#Creating database for reviews beauty file
rbf = client.dsbc.rbf
with open('reviews_Beauty.json') as reviews_beauty_file:
    for line in reviews_beauty_file:
        review = json.loads(line)  
        rbf.insert(review)
        

In [11]:
#new database for selective items
ndb = client.dsbc.ndb

In [22]:
match_list = rbf.find({"asin":{"$in": [id_list.keys()[3]]}})

In [24]:
id_list["B001JQLNMI"]

[u'Olay Professional Pro-X Hydra Firming Cream Anti Aging 1.7 Oz', 15.38]

In [25]:
product_reviews = {}
product_reviews["B001JQLNMI"]=[i for i in match_list]


In [39]:
simple_review = product_reviews["B001JQLNMI"][0]['reviewText']

In [94]:
from nltk.book import *

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [114]:
saying = ['after','all','is','said','and','done',
          'more','is','said','than','done']

In [131]:
saying = "After all is said and done, more is said than done"
saying.endswith("n")

False

(['after', 'all', 'and', 'done', 'is', 'more', 'said', 'than'],
 ['said', 'than'])

# RATING ALGORITHM

In [65]:
def amazon_rating(x1,x2,x3,x4,x5):
    return (x1+2*x2+3*x3+4*x4+5*x5)/float(x1+x2+x3+x4+x5)

In [66]:
print "50 shades rating", amazon_rating(7708,2732,2970,3584,14876)
print "twightlight rating", amazon_rating(802,349,503,781,4488)
print "the hobbit rating", amazon_rating(141,104,265,1071,6619)
print "daughter of the forest", amazon_rating(20,20,19,73,348)
print "best seller of 2014", amazon_rating(100,108,166,414,1745)
print "divergent", amazon_rating(411,519,1316,3918,14375)
print "eragon", amazon_rating(443,267,331,572,2019)
print "to kill a mocking bird", amazon_rating(141,96,184,657,4708)

50 shades rating 3.47656102918
twightlight rating 4.12725696952
the hobbit rating 4.69792682927
daughter of the forest 4.47708333333
best seller of 2014 4.41966048164
divergent 4.52524465651
eragon 3.95181718062
to kill a mocking bird 4.67559626685


In [67]:
def smooth_rating(x1,x2,x3,x4,x5,alpha,beta):
    return ((x1+2*x2+3*x3+4*x4+5*x5)+alpha*beta)/(float(x1+x2+x3+x4+x5)+beta)

In [68]:
print "50 shades rating", smooth_rating(7708,2732,2970,3584,14876,3,100)
print "twightlight rating", smooth_rating(802,349,503,781,4488,3,100)
print "the hobbit rating", smooth_rating(141,104,265,1071,6619,3,100)
print "daughter of the forest", smooth_rating(20,20,19,73,348,3,100)
print "best seller of 2014", smooth_rating(100,108,166,414,1745,3,100)
print "divergent", smooth_rating(411,519,1316,3918,14375,3,100)
print "eragon", smooth_rating(443,267,331,572,2019,3,100)
print "to kill a mocking bird", smooth_rating(141,96,184,657,4708,3,100)

50 shades rating 3.47507037848
twightlight rating 4.11120603731
the hobbit rating 4.67746987952
daughter of the forest 4.2224137931
best seller of 2014 4.36574249905
divergent 4.51785454722
eragon 3.92631296892
to kill a mocking bird 4.64712878016


In [69]:
def weight_extreme_rating(x1,x2,x3,x4,x5,alpha,beta):
    numer = (2*x1+2*x2+3*x3+4*x4+10*x5)+alpha*beta
    denom = float(2*x1+x2+x3+x4+2*x5+beta)
    return numer/denom

In [70]:
print "50 shades rating", weight_extreme_rating(7708,2732,2970,3584,14876,3,100)
print "twightlight rating", weight_extreme_rating(802,349,503,781,4488,3,100)
print "the hobbit rating", weight_extreme_rating(141,104,265,1071,6619,3,100)
print "daughter of the forest", weight_extreme_rating(20,20,19,73,348,3,100)
print "best seller of 2014", weight_extreme_rating(100,108,166,414,1745,3,100)
print "divergent", weight_extreme_rating(411,519,1316,3918,14375,3,100)
print "eragon", weight_extreme_rating(443,267,331,572,2019,3,100)
print "to kill a mocking bird", weight_extreme_rating(141,96,184,657,4708,3,100)

50 shades rating 3.54118854713
twightlight rating 4.23251847641
the hobbit rating 4.78479415671
daughter of the forest 4.43987341772
best seller of 2014 4.53774006253
divergent 4.67268877911
eragon 4.06700032289
to kill a mocking bird 4.75398230088


In [72]:
def strict_rating(x1,x2,x3,x4,x5,alpha,beta):
    numer = (2*x1+4*x2+3*x3+4*x4+5*x5)+alpha*beta
    denom = float(2*x1+x2+x3+x4+x5+beta)
    return numer/denom

In [79]:
print "50 shades rating", strict_rating(7708,2732,2970,3584,14876,3,100)
print "twightlight rating", strict_rating(802,349,503,781,4488,3,100)
print "the hobbit rating", strict_rating(141,104,265,1071,6619,3,100)
print "daughter of the forest", strict_rating(20,20,19,73,348,3,100)
print "best seller of 2014", strict_rating(100,108,166,414,1745,3,100)
print "divergent", strict_rating(411,519,1316,3918,14375,3,100)
print "eragon", strict_rating(443,267,331,572,2019,3,100)
print "to kill a mocking bird", strict_rating(141,96,184,657,4708,3,100)

50 shades rating 2.7851686152
twightlight rating 3.61365480468
the hobbit rating 4.58086693079
daughter of the forest 4.07903225806
best seller of 2014 4.20437698553
divergent 4.43148035972
eragon 3.48051104374
to kill a mocking bird 4.51151102464


In [88]:
print strict_rating(1,0,3,32,193,2.5,100)
print strict_rating(0,0,1,6,104,2.5,100)
print strict_rating(0,1,7,31,230,2.5,100)
print strict_rating(253,184,297,590,2428,2.5,100)
print strict_rating(101,64,117,205,895,2.5,100)

4.06906906907
3.77725118483
4.19783197832
3.52302631579
3.42258748674


In [87]:
def strict_rating(x1,x2,x3,x4,x5,alpha,beta):
    numer = (3*x1+4*x2+3*x3+4*x4+5*x5)+alpha*beta
    denom = float(5*x1+x2+x3+x4+x5+beta)
    return numer/denom

In [90]:
stopwords = nltk.corpus.stopwords.words('english')

In [93]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')